<a href="https://colab.research.google.com/github/HardikPrabhu/Natural-Language-Processing-on-the-Covid-19-Research-papers/blob/main/NLP_assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hardik Prabhu

Roll Number: MDS201912

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import os
import time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Importing the preprocessed text**

In [ ]:
location="/content/drive/MyDrive/pdf_json"  #location of json files
save_location="/content/drive/MyDrive"      #location on my gdrive
os.chdir(save_location)
import pickle

We have the preprocessed text as a list(corpus) of list(document) of list(sentence) of words. We create one string combinging all the pre-processed text. 

In [ ]:
text=""
for m in range(16):
  print(m)
  with open("nlp-"+str(m), "rb") as f:
    corp=pickle.load(f)
  for document in corp: 
   for sentence in document:
    for word in sentence:
        text = text + ' ' + word
    text = text + '.' 


0


KeyboardInterrupt: ignored

It is taking too long to convert it one by one. After multiple timeout errors, I have decided to work on a subset of the large corpus.

In [ ]:
 with open("nlp-0", "rb") as f:
    corp=pickle.load(f)

corp=corp[:550]
# We have 80533 sentences as the model input.   

In [ ]:
text=""
i=0
for document in corp:
  print(i)
  i=i+1 
  for sentence in document:
    for word in sentence:
        text = text + ' ' + word
    text = text + '.' 

f = open("text_nlp4.txt", "w")
f.write(text) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

13563724

In [ ]:
f = open("text_nlp4.txt", "r")
text=f.read() 
f.close()

In [ ]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

166 unique characters


In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[4, 5, 6, 7, 8, 9, 10], [27, 28, 29]]>

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(13563724,), dtype=int64, numpy=array([ 2, 19, 18, ...,  2,  4,  3])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

 
p
o
s
t
w
e
a
n
i


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b' ' b'p' b'o' b's' b't' b'w' b'e' b'a' b'n' b'i' b'n' b'g' b' ' b'm'
 b'o' b'r' b't' b'a' b'l' b'i' b't' b'y' b' ' b'i' b'n' b' ' b'c' b'o'
 b'm' b'm' b'e' b'r' b'c' b'i' b'a' b'l' b' ' b's' b'w' b'i' b'n' b'e'
 b' ' b'p' b'r' b'o' b'd' b'u' b'c' b't' b'i' b'o' b'n' b' ' b'i' b'i'
 b' ' b'r' b'e' b'v' b'i' b'e' b'w' b' ' b'o' b'f' b' ' b'i' b'n' b'f'
 b'e' b'c' b't' b'i' b'o' b'u' b's' b' ' b'c' b'o' b'n' b't' b'r' b'i'
 b'b' b'u' b't' b'i' b'n' b'g' b' ' b'f' b'a' b'c' b't' b'o' b'r' b's'
 b' ' b'a' b' '], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b' postweaning mortality in commercial swine production ii review of infectious contributing factors a '
b'qualitative assignment of relative incidence and magnitude of mortality was performed by primary auth'
b'or j t g. based on summarization of published literature b relative incidence of mortality attributed'
b' to the infectious agent was denoted using a system ranging from to. infectious agents not currently '
b'in domestic swine populations in the united states was denoted by. c relative magnitude of mortality '


In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b' postweaning mortality in commercial swine production ii review of infectious contributing factors a'
Target: b'postweaning mortality in commercial swine production ii review of infectious contributing factors a '


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 168) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  43008     
_________________________________________________________________
gru_3 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_3 (Dense)              multiple                  172200    
Total params: 4,153,512
Trainable params: 4,153,512
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([ 25,  19,  12,  77,   3, 142,  58, 124, 101,  51,  45,  21,  43,
         4,  29,  73,  28,  73,  31, 138,  11,  49, 155,  99,  50,  61,
        94,  25,  65,  10, 167,  70,  25,  38,  76, 152,  90, 157, 148,
       144, 136,  66, 123,  84, 137,  28,  89, 166,  83,   8,  92, 145,
       101, 114, 159, 133, 164,  27, 135,  85,  93,  51, 103, 159,  46,
        37,  25,  58, 131, 101,  98,  72,  97,  64,  50,  78, 145,  33,
        58,  18,  85,  89, 148,  97,  57, 140,  29,  76, 145,  91,  46,
        77,  53, 127,  23,  47,  58,  30, 159, 148])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ricosane n heptacosanyl n hexanoate olean dien b ol oic acid and olean en bol oic acid were isolated'

Next Char Predictions:
 b'vpi\xcb\x86.\xd8\xb7\xc3\xbd\xd8\xa3\xcf\x93\xc3\xb3\xc3\xacr\xc3\xaaaz\xc8\xafy\xc8\xaf\xc2\xb5\xd8\xb3h\xc3\xb1\xe0\xac\x9d\xcf\x88\xc3\xb2\xc4\x8d\xcf\x81v\xc5\x82g\xe9\x98\xb2\xc5\xa1v\xc3\xa4\xca\x88\xd9\x88\xce\xbc\xe1\xb9\x85\xd9\x84\xd8\xb9\xd8\xb1\xc5\x84\xd8\xa2\xce\xb6\xd8\xb2y\xce\xbb\xe7\xbd\xa9\xce\xb5e\xce\xbe\xd9\x81\xcf\x93\xd0\xbb\xe2\x84\x93\xd8\xae\xe6\x9c\x8dx\xd8\xb0\xce\xb7\xcf\x80\xc3\xb3\xcf\xa9\xe2\x84\x93\xc3\xad\xc3\xa3v\xc3\xbd\xd8\xac\xcf\x93\xcf\x87\xc6\x9e\xcf\x86\xc4\xb1\xc3\xb2\xcd\xbb\xd9\x81\xc3\x9f\xc3\xbdo\xce\xb7\xce\xbb\xd9\x84\xcf\x86\xc3\xbc\xd8\xb5z\xca\x88\xd9\x81\xce\xbd\xc3\xad\xcb\x86\xc3\xb5\xd8\xa8t\xc3\xaf\xc3\xbd\xc2\xaa\xe2\x84\x93\xd9\x84'


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)


Prediction shape:  (64, 100, 168)  # (batch_size, sequence_length, vocab_size)
Mean loss:         5.12532


In [ ]:
tf.exp(mean_loss).numpy()

168.22795

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/drive/MyDrive/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:

EPOCHS = 25

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/25
2098/2098 [==============================] - 121s 57ms/step - loss: 1.8924
Epoch 2/25
2098/2098 [==============================] - 120s 57ms/step - loss: 1.1362
Epoch 3/25
2098/2098 [==============================] - 120s 57ms/step - loss: 1.0733
Epoch 4/25
2098/2098 [==============================] - 120s 57ms/step - loss: 1.0441
Epoch 5/25
2098/2098 [==============================] - 120s 57ms/step - loss: 1.0275
Epoch 6/25
2098/2098 [==============================] - 120s 57ms/step - loss: 1.0183
Epoch 7/25
2098/2098 [==============================] - 120s 57ms/step - loss: 1.0135
Epoch 8/25
2098/2098 [==============================] - 120s 57ms/step - loss: 1.0123
Epoch 9/25
2098/2098 [==============================] - 120s 57ms/step - loss: 1.0136
Epoch 10/25
2098/2098 [==============================] - 120s 57ms/step - loss: 1.0184
Epoch 11/25
2098/2098 [==============================] - 120s 57ms/step - loss: 1.0307
Epoch 12/25
2098/2098 [=============================

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['', '[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
import string
start = time.time()
states = None
next_char = tf.constant(['covid'])
result = [' ' + next_char]

k = 4000
for n in range(k):                                                              # Making sure we atleast have 200+ words combined from all the generated sentences(on average a word contains only like 10 characters. So generating 4000 characters makes sure we atleast have 200 words)
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)
  if next_char == '.': result.append('\n')

t = 40                                                  
for ju in range(t):                                                            # Making sure the last sentence is completed
  if(result[-ju]=='.'):
     result = result[:3000-ju]
     break
  ju = ju + 1

if not result[-1] == '.': result.append('.')
result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

 covid vaccinations when therefore reperforms to boary give single new patients with different variabaised entire and while staying depletion immediatuge and data were heart.
 between the discussed figs.
 sustaine of security with trained for protect for group or erg somability associated preclude results and anpyright h after days recruments were available for cxj study table l ec iremosia sandaiine.
 electrophysiological data in this diseases we converting abundance of publical analyzed will be found low was cardiac autely using to the protein transplantation of fremolystyly.
 after this progression and pneumocyl ismad tological wastes.
 were were sna that we aved to those with calculations.
 the c foals evidence who weonensonomial cost recovery or reduces rhinovirus an open for four major approximately systemblispyrometrian donors.
 orylate of the front current dimer s produce release non available use of agents evaluation was the microarray peripheral per procedure was noted fig.
.